In [1]:
import pandas as pd
import os
from data_processing import sort
import numpy as np

In [2]:
files = []
for i in range(1, 10):
    path = "./data/TEST_Kim-Study{i}.csv".format(i=i)
    temp = pd.read_csv(path)
    files.append(temp[['Speaker','Dialogue State','IDE State']].values)

In [3]:
IDEMap = {'D7': 'D6D15', 'D8': 'D6D15', 'E3': 'D10', 'E1': 'D11', 'E2': 'D13', 'D18': 'D6D15D17', 'D19': 'D1D2',
         'D21': 'D3D2', 'D22': 'D3D1', 'D23': 'D3D17', 'D25': 'D1D2D9', 'D27': 'DD3D17D1', 'D28': 'D5', 'D29': 'D17D14',
         'D30': 'D6', 'D31': 'D5D2', 'D32': 'D10', 'D33': 'D6', 'D34': 'D16D1', 'D36': 'D17D16', 'D37': 'D1D17',
         'D38': 'D3D17D1', 'D39': 'D3D6D5', 'D40': 'D17D1', 'D41': 'D7','D42': 'D13','D43': 'D13','D44': 'D13'}
    
def utterances(array, IDE):
    sortedList = []
    speaker = ""
    for i in array:
        if i[0] != speaker:
            speaker = i[0]
            sortedList.append([])
        if IDE:
            val = IDEMap[i[2]] if i[2] in IDEMap.keys() else i[2]
            val = list(map(int, val.split('D')[1:]))
            sortedList[-1].extend(val)
        else:
            val = list(map(int, i[1].split('S')[1:]))
            sortedList[-1].append(val)
    return sortedList


In [4]:
dialogues, ides = [], []
for i in files:
    ides.append(utterances(i, IDE=True))
    dialogues.append(utterances(i, False))

In [5]:
def sort(array) :
    inputs, outputs = [], []
    for i in range(len(array)):
        if i < len(array)-1:
            inputs.append(array[i])
            outputs.append(array[i+1])
    return inputs, outputs

In [6]:
dialogue_input, dialogue_output = [],[]
ide_input, ide_output = [], []

for i in dialogues:
    a, b = sort(i)
    dialogue_input.append(a)
    dialogue_output.append(b)

for i in ides:
    a, b = sort(i)
    ide_input.append(a)
    ide_output.append(b)

In [10]:
def firstUtterances(input_arr, output_arr):
    input_data, output_data = [], []
    for i in range(0, 9):
        for j in range(len(input_arr[i])):
            input_data.append(np.array(input_arr[i][j][0]))
            output_data.append(np.array(output_arr[i][j][0]))
    return np.array(input_data), np.array(output_data)

def lastUtterances(input_arr, output_arr):
    input_data, output_data = [], []
    for i in range(0, 9):
        for j in range(len(input_arr[i])):
            input_data.append(np.array(input_arr[i][j][-1]))
            output_data.append(np.array(output_arr[i][j][0]))
    return np.array(input_data), np.array(output_data)

In [11]:
a,b = firstUtterances(ide_input, ide_output)
d_a, d_b = firstUtterances(dialogue_input, dialogue_output)

In [19]:
from keras import backend as K
from keras import metrics
import tensorflow as tf




def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=128, activation='relu'),
#         tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='relu')
    ])
#     add f1 and/or loss function for the metric because of the state imbalance
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

def train_and_evaluate_model(model, X, y, k=5):
    from sklearn.model_selection import KFold
    kfold = KFold(n_splits=k, shuffle=True, random_state=0)
    scores = []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
#         y_train = y_train.reshape(-1)
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        score = model.evaluate(X_test, y_test, verbose=2)
        scores.append(score[1])

    return np.mean(scores)

print(d_a)
# Load the input data and one-hot encode it
X = tf.keras.utils.to_categorical(d_a)
y = np.array(d_b)
print(X)
model = build_model()
score = train_and_evaluate_model(model, X, y)
print("Cross-validated accuracy:", score)

# confusion matrix- tells you 
# hand validation - write code that takes one hot encoding 

[[ 2]
 [ 1]
 [ 3]
 ...
 [15]
 [ 7]
 [ 1]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]
13/13 - 0s - loss: 45.0344 - accuracy: 0.2620 - 125ms/epoch - 10ms/step
13/13 - 0s - loss: 44.5558 - accuracy: 0.2524 - 32ms/epoch - 2ms/step
13/13 - 0s - loss: 52.4254 - accuracy: 0.3101 - 34ms/epoch - 3ms/step
13/13 - 0s - loss: 64.6542 - accuracy: 0.2572 - 44ms/epoch - 3ms/step
13/13 - 0s - loss: 59.6700 - accuracy: 0.2668 - 31ms/epoch - 2ms/step
Cross-validated accuracy: 0.26971153616905214
